<a href="https://colab.research.google.com/github/jcandane/CI_Theory/blob/main/RASCI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import packages

In [ ]:
import numpy as np
from itertools import combinations, permutations

π = np.pi
α = 0.007297352
c = 1.0/α
np.set_printoptions(precision=4, linewidth=200, threshold=2000, suppress=True)

### for proformance reasons, list of determinants should be in np.int8 (1 B, upto 127 dets), np.int16 (2 B, upto 32,767 dets)), np.int32 (4 B, upto 2 billon dets)
CI_dt  = np.int16 ### index that labels CI states, i.e. I, J, K in notes

### sample references

In [ ]:
ref1_sp = np.array([[1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 0, 0, 0]], dtype=np.int8)
ref2_sp = np.array([[0, 1, 0, 1, 1, 0, 0], [0, 1, 1, 0, 0, 1, 0]], dtype=np.int8)
ref3_sp = np.array([[1, 1, 1, 0, 1, 0, 0, 1, 0], [0, 1, 1, 1, 1, 1, 0, 0, 0]], dtype=np.int8)
ref4_sp = np.array([[1, 1, 0, 0, 0], [1, 1, 0, 0, 0]], dtype=np.int8)
ref5_sp = np.array([[[1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 0, 0, 0]], [[1, 1, 0, 1, 1, 0, 1], [0, 1, 1, 1, 0, 1, 0]]], dtype=np.int8)
ref6_sp = np.array([[[1, 1, 0, 0, 0], [1, 1, 0, 0, 0]], [[1, 0, 0, 1, 0], [1, 1, 0, 0, 0]], [[1, 1, 0, 0, 0], [1, 0, 0, 0, 1]]], dtype=np.int8)

## General Defintions

In [ ]:
def Ext(A, N, numtype=np.int8):
    if A.ndim == 1:
        return np.einsum("I, j -> Ij", A, np.ones( N, dtype=numtype))
    if A.ndim == 2:
        return np.einsum("uI, j -> uIj", A, np.ones( N, dtype=numtype))

def ΛMOgetB(Λ, N_mo):
  "Given Λ (i occupied orbitals for each determinant) get B (binary rep.)"

  Binary  = np.zeros((Λ.shape[0], N_mo), dtype=np.int8)
  for I in range(len(Binary)):
      Binary[I, Λ[I,:]] = 1

  return Binary

def SpinOuterProduct(A, B, stack=False):
  ΛA = np.einsum("Ii, J -> IJi", A, np.ones(B.shape[0], dtype=np.int8)).reshape( (A.shape[0]*B.shape[0], A.shape[1]) )
  ΛB = np.einsum("Ii, J -> JIi", B, np.ones(A.shape[0], dtype=np.int8)).reshape( (A.shape[0]*B.shape[0], B.shape[1]) )
  
  if stack:
    return np.array([ΛA,ΛB])
  else:
    return ΛA, ΛB

def combine_arrays(Λ_sample, V_sample):

    if V_sample.size == 0:
        return Λ_sample

    else:
        l = V_sample.shape[1] + Λ_sample.shape[1] # length of new array
        m = V_sample.shape[1] # level of excitation
        a_new = np.empty((V_sample.shape[0] * Λ_sample.shape[0], V_sample.shape[1] + Λ_sample.shape[1] ), dtype=np.int8)
        a_new[:,:l - m ] = np.repeat(Λ_sample, len(V_sample), axis=0)
        a_new[:, l - m:] = np.repeat(V_sample, len(Λ_sample), axis=1).reshape((len(Λ_sample)*len(V_sample), V_sample.shape[1]), order="F")

        return np.sort(a_new, axis=1)

def CIunique(dets):
    BB = np.unique( ( dets.swapaxes(0,1)).reshape(dets.shape[1], dets.shape[0]*dets.shape[2], order="C") , axis=0)
    return BB.reshape(BB.shape[0], 2, BB.shape[1]//2, order="C").swapaxes(0,1)

def TEST_electronnumber(AA): ## with B_sIp
    arr = np.einsum("sIp -> I", AA)
    if np.all(arr == arr[0]):
        return True
    else:
        return arr

def TEST_unquineness(AA): ## with B_sIp
    return AA.shape[1] == len( np.unique( (AA.swapaxes(0,1)).reshape((AA.shape[1], 2*AA.shape[2])) ,axis=1 ) )

## MO Excitation

In [ ]:
def MO_X(MO_p, m, SF=False):
    """ 
    m-level CI excitations on an MO 
    m=0 should return the reference
    """

    occ = np.asarray([ np.where(MO_p == 1)[0] ]) ## occupied space
    vac = np.asarray([ np.where(MO_p == 0)[0] ]) ## vacant   space

    if SF: ### do SF-calculation
        if m < 0 and abs(m) <= len(occ[0]): ## if m is negative, then we do all combinations deleting occupied orbital(s) 
            Occ_choices = np.asarray( list(combinations(  occ[0], len(occ[0]) + m ) ) , dtype=np.int8)
            return ΛMOgetB(Occ_choices, len(MO_p))
        if m == 0: ## if m = 0 , return reference
            return np.asarray([MO_p])
        if m > 0 and m <= len(vac[0]): ## if m is positive, we will do all combinations by adding occupied orbital(s) 
            Vac_choices = np.asarray( list(combinations(  vac[0], m ) ) , dtype=np.int8)
            return ΛMOgetB(combine_arrays(occ, Vac_choices), len(MO_p))
        else:
            return None

    else: ### MO-conserved-calculation
        ## excitation order must be less than avaiable orbitals
        if m <= len(occ[0]) and m <= len(vac[0]): 
            Occ_choices = np.asarray( list(combinations(  occ[0], len(occ[0]) - m ) ) , dtype=np.int8)
            Vac_choices = np.asarray( list(combinations(  vac[0], m ) ) , dtype=np.int8)
            return ΛMOgetB(combine_arrays(Occ_choices, Vac_choices), len(MO_p))
        else:
            return None

reff = np.array([1, 1, 1, 0, 1, 0, 0, 1, 0, 0]) ## ref3_sp[0] #
print(reff)
MOexcite = MO_X(reff, 0, SF=True)

### Test 1
print( np.all(np.einsum("Ip -> I", MOexcite) == np.sum(reff)) ) ## 5 electrons

### Test 2
print( MOexcite.shape[0] == (np.unique( MOexcite, axis=1)).shape[0] )


print( MO_X(reff, -6, SF=True) )

[1 1 1 0 1 0 0 1 0 0]
True
True
None


## SO Excitation

In [ ]:
def SO_X(ref_SO, m, SF=False):

    if SF:
        C = None
        D = None
        ## +3 & -3
        A = MO_X(ref_SO[0],  m, SF=True)
        B = MO_X(ref_SO[1], -m, SF=True)
        if A is not None and B is not None:
            C = SpinOuterProduct(A, B, stack=True)
        
        ## -3 & +3
        A = MO_X(ref_SO[0], -m, SF=True)
        B = MO_X(ref_SO[1],  m, SF=True)
        if A is not None and B is not None:
            D = SpinOuterProduct(A, B, stack=True)
        
        if C is not None and D is not None:
            return np.append(D, C, axis=1)
        if C is not None:
            return C
        if D is not None:
            return D
        else:
            return None

    else:
        D     = None
        first = True
        for j in range(m+1):
            A = MO_X(ref_SO[0],   j)
            B = MO_X(ref_SO[1], m-j)
            if A is not None and B is not None:
                C = SpinOuterProduct(A, B, stack=True)
                if first:
                    D = C
                    first = False
                else:
                    D = np.append(D, C, axis=1)
            else:
                continue
        return D

SO_X(ref4_sp, 2, SF=False)

array([[[1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 1, 0],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 1, 0, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 1, 0],


## Reference Excitation

In [ ]:
CIm = [0,1,2]
SFm = [1,3]

def Ref_X(ref_SO, RAS=None, CIm=None, SFm=None):

    if RAS is None:
        RAS = np.arange(0, ref_SO.shape[1], 1, dtype=int)

    RASref = ref_SO[:, RAS]
    #loop   = Ext(reference, RASCI.shape[1]).swapaxes(1,2)
    

    ### CIm Loop
    outCII = None
    if CIm is not None:
        first = True
        for i in range(len(CIm)):
            CIm_sIp = SO_X(RASref, CIm[i], SF=False)
            if first and CIm_sIp is not None:
                outCI = CIm_sIp
                first = False
                continue
            if CIm_sIp is not None:
                outCI = np.append(outCI, CIm_sIp, axis=1)
            else:
                continue
        ## refit with RAS 
        outCII = Ext(ref_SO, outCI.shape[1]).swapaxes(1,2)
        outCII[:,:,RAS] = outCI

    ### SFm Loop
    outSFF = None
    if SFm is not None:
        first = True
        for i in range(len(SFm)):
            SFm_sIp = SO_X(RASref, SFm[i], SF=True)
            if first and SFm_sIp is not None:
                outSF = SFm_sIp
                first = False
                continue
            if SFm_sIp is not None:
                outSF = np.append(outSF, SFm_sIp, axis=1)
            else:
                continue
        ## refit with RAS 
        outSFF = Ext(ref_SO, outSF.shape[1]).swapaxes(1,2)
        outSFF[:,:,RAS] = outSF

    return outCII, outSFF

outCI, outSF = Ref_X(ref4_sp, CIm=CIm, SFm=SFm)

print( TEST_electronnumber(outCI) )
print( outCI.shape[1] == CIunique(outCI).shape[1] )

True
True


## Multireference Excitation

In [ ]:
print( ref4_sp.shape ) ## Single Refernce
print( ref6_sp[0].shape ) ## Multireference

outCI, outSF = Ref_X(ref6_sp[0], RAS=None, CIm=[1], SFm=None)

(2, 5)
(2, 5)


In [ ]:
def MR_X(MR_SO, RAS=None, CIm=None, SFm=None):

    firstCI = True
    firstSF = True
    outputCI = None
    outputSF = None

    if RAS is not None: ## checking RAS
        if np.max(RAS) > MR_SO.shape[-1]:
            RAS = None

    if MR_SO.ndim == 2:
        outCI, outSF = Ref_X(MR_SO, RAS=RAS, CIm=CIm, SFm=SFm)

        if outCI is not None and firstCI:
            outputCI = outCI
            firstCI  = False
        if outCI is not None:
            outputCI = np.append(outputCI, outCI, axis=1)
            outputCI = CIunique(outputCI)
        if outSF is not None and firstSF:
            outputSF = outSF
            firstSF  = False
        if outSF is not None:
            outputSF = np.append(outputSF, outSF, axis=1)
            outputSF = CIunique(outputSF)

    if MR_SO.ndim == 3:

        for i in range(len(MR_SO)):
            outCI, outSF = Ref_X(MR_SO[i], RAS=RAS, CIm=CIm, SFm=SFm)

            if outCI is not None and firstCI:
                outputCI = outCI
                firstCI  = False
            if outCI is not None:
                outputCI = np.append(outputCI, outCI, axis=1)
                outputCI = CIunique(outputCI)
            if outSF is not None and firstSF:
                outputSF = outSF
                firstSF  = False
            if outSF is not None:
                outputSF = np.append(outputSF, outSF, axis=1)
                outputSF = CIunique(outputSF)

    return outputCI, outputSF

CI, SF = MR_X(ref4_sp, RAS=[2,3,4], CIm=[0,1], SFm=None)
CI, SF = MR_X(ref6_sp, RAS=[2,3,4,5,6], CIm=[0,1], SFm=[0,1,2,20]) ## ref3_sp ref6_sp
CI.shape

(2, 28, 5)

## Tests

### Test the Electron Number (electron number should be conserved)

In [ ]:
print( (np.diff( np.einsum("sIp -> I", CI) ) == 0).all() )
print( (np.diff( np.einsum("sIp -> I", SF) ) == 0).all() )

True
True


### Uniqueness (Check that all determinants are different)

In [ ]:
def CIuniqueX(dets):
    out = np.unique( ( dets ).reshape(dets.shape[0], dets.shape[1]*dets.shape[2], order="C") , axis=0)
    return out.reshape(out.shape[0], 2, out.shape[1]//2, order="C")

dets = CI.swapaxes(0,1)
print( (dets).shape[0] == (CIuniqueX(dets)).shape[0] )

True


# Slater Condon Rules

In [ ]:
def givenBgetΛ(B):
  "Given B (entire MO binary occupation) get Λ (i occupied)"

  numA = len( (B[0,0])[ B[0,0] == 1 ] ) ## count num. of A occupied
  numB = len( (B[0,1])[ B[0,1] == 1 ] ) ## count num. of B occupied

  ΛA = np.zeros((B.shape[0], numA), dtype=np.int8)
  ΛB = np.zeros((B.shape[0], numB), dtype=np.int8)
  for I in range(len(B)):
    ΛA[I] = np.where(B[I,0] == 1)[0]
    ΛB[I] = np.where(B[I,1] == 1)[0]

  return ΛA, ΛB

In [ ]:
def SlaterCondon(Binary):
    N_s    = np.einsum("sp -> s", Binary[0])
    Difference     = np.einsum("Isp, J -> IJsp", Binary, np.ones(len(Binary), dtype=np.int8)) - np.einsum("Isp, J -> JIsp", Binary, np.ones(len(Binary), dtype=np.int8))
    Sum            = np.einsum("Isp, J -> IJsp", Binary, np.ones(len(Binary), dtype=np.int8)) + np.einsum("Isp, J -> JIsp", Binary, np.ones(len(Binary), dtype=np.int8))
    SpinDifference = np.einsum("IJsp -> IJs", np.abs(Difference))//2

    ##  indices for 1-difference
    I_A, J_A = np.where( np.all(SpinDifference==np.array([1,0], dtype=CI_dt), axis=2) )
    I_B, J_B = np.where( np.all(SpinDifference==np.array([0,1], dtype=CI_dt), axis=2) )
    I_A = I_A.astype(CI_dt)
    J_A = J_A.astype(CI_dt)
    I_B = I_B.astype(CI_dt)
    J_B = J_B.astype(CI_dt)

    A_t  = (np.where( Difference[I_A, J_A, 0] ==  1 )[1]).astype(CI_dt)
    A    = (np.where( Difference[I_A, J_A, 0] == -1 )[1]).astype(CI_dt)
    B_t  = (np.where( Difference[I_B, J_B, 1] ==  1 )[1]).astype(CI_dt)
    B    = (np.where( Difference[I_B, J_B, 1] == -1 )[1]).astype(CI_dt)
    CA_i = ((np.where( Sum[I_A, J_A, 0] == 2 )[1]).reshape(len(I_A), N_s[0]-1)).astype(CI_dt)
    CB_i = ((np.where( Sum[I_B, J_B, 1] == 2 )[1]).reshape(len(I_B), N_s[1]-1)).astype(CI_dt)

    ## indices for 2-differences
    I_AA, J_AA = np.where( np.all(SpinDifference==np.array([2,0], dtype=CI_dt), axis=2) )
    I_BB, J_BB = np.where( np.all(SpinDifference==np.array([0,2], dtype=CI_dt), axis=2) )
    I_AB, J_AB = np.where( np.all(SpinDifference==np.array([1,1], dtype=CI_dt), axis=2) )
    I_AA = I_AA.astype(CI_dt)
    J_AA = J_AA.astype(CI_dt)
    I_BB = I_BB.astype(CI_dt)
    J_BB = J_BB.astype(CI_dt)
    I_AB = I_AB.astype(CI_dt)
    J_AB = J_AB.astype(CI_dt)

    AA   = (np.where( Difference[I_AA, J_AA, 0] == -1)[1].reshape(len(I_AA),2).T).astype(CI_dt)
    AA_t = (np.where( Difference[I_AA, J_AA, 0] ==  1)[1].reshape(len(I_AA),2).T).astype(CI_dt)
    BB   = (np.where( Difference[I_BB, J_BB, 1] == -1)[1].reshape(len(I_BB),2).T).astype(CI_dt)
    BB_t = (np.where( Difference[I_BB, J_BB, 1] ==  1)[1].reshape(len(I_BB),2).T).astype(CI_dt)
    AB   = np.asarray([ np.where( Difference[I_AB, J_AB, 0] == -1 )[1], np.where( Difference[I_AB, J_AB, 1] == -1 )[1] ], dtype=CI_dt)
    AB_t = np.asarray([ np.where( Difference[I_AB, J_AB, 0] ==  1 )[1], np.where( Difference[I_AB, J_AB, 1] ==  1 )[1] ], dtype=CI_dt)

    ## get orbital occupations for each up-xor-down
    ΛA, ΛB = givenBgetΛ(Binary)

    ## get sign
    sign  = np.cumsum( Binary, axis=2)
    for I in range(len(Binary)):
        sign[I, 0, ΛA[I]] = np.arange(0, N_s[0], 1)
        sign[I, 1, ΛB[I]] = np.arange(0, N_s[1], 1)

    Γ_Isp = ( (-1)**(sign) ).astype(np.int8)

    return [I_A, J_A, A, A_t, I_B, J_B, B, B_t, CA_i, CB_i], [I_AA, J_AA, AA, AA_t, I_AB, J_AB, AB, AB_t, I_BB, J_BB, BB, BB_t], Γ_Isp

SC1, SC2, sign = SlaterCondon(dets)

## Now Lets get the RASCI Hamiltonian

In [ ]:
Γ_Isp = sign
I_A, J_A, A, A_t, I_B, J_B, B, B_t, CA_i, CB_i = SC1
I_AA, J_AA, AA, AA_t, I_AB, J_AB, AB, AB_t, I_BB, J_BB, BB, BB_t = SC2

## SC0 & SC3
AB = Ext(ΛA, N_s[1])
BA = Ext(ΛB, N_s[0])
AA = Ext(ΛA, N_s[0])
BB = Ext(ΛB, N_s[1])

Rule0  = np.einsum("Ii  -> I", H1[0, ΛA, ΛA]) 
Rule0 += np.einsum("Ii  -> I", H1[1, ΛB, ΛB])
Rule0 += np.einsum("Iij -> I", H2[0, 0, AA, AA, AA.swapaxes(1,2), AA.swapaxes(1,2)])/2
Rule0 += np.einsum("Iij -> I", H2[1, 0, BA, BA, AB.swapaxes(1,2), AB.swapaxes(1,2)])/2
Rule0 += np.einsum("Iij -> I", H2[0, 1, AB, AB, BA.swapaxes(1,2), BA.swapaxes(1,2)])/2
Rule0 += np.einsum("Iij -> I", H2[1, 1, BB, BB, BB.swapaxes(1,2), BB.swapaxes(1,2)])/2

H_CI = np.diag(Rule0)

## SC1
if I_A.size > 0:
  H_CI[I_A, J_A]  = H1[0, A, A_t]
  H_CI[I_A, J_A] += np.einsum("In -> I", H2[0, 0, Ext(A, N_s[0]-1), Ext(A_t, N_s[0]-1), CA_i, CA_i])
  H_CI[I_A, J_A] += np.einsum("Ii -> I", H2[0, 1, Ext(A, N_s[1]  ), Ext(A_t, N_s[1]  ), ΛB[I_A], ΛB[J_A]])
  H_CI[I_A, J_A] *= Γ_Isp[I_A, 0, A_t] * Γ_Isp[J_A, 0, A]

if I_B.size > 0:
  H_CI[I_B, J_B]  = H1[1, B, B_t]
  H_CI[I_B, J_B] += np.einsum("In -> I", H2[1, 1, Ext(B, N_s[1]-1), Ext(B_t, N_s[1]-1), CB_i, CB_i])
  H_CI[I_B, J_B] += np.einsum("Ii -> I", H2[1, 0, Ext(B, N_s[0]  ), Ext(B_t, N_s[0]  ), ΛA[I_B], ΛA[J_B]])
  H_CI[I_B, J_B] *= Γ_Isp[I_B, 1, B_t] * Γ_Isp[J_B, 1, B]

## SC2
if I_AA.size > 0: 
    H_CI[I_AA, J_AA]  = H2[0, 0, AA[0], AA_t[0], AA[1], AA_t[1]]
    H_CI[I_AA, J_AA] *= Γ_Isp[I_AA, 0, AA[0]] * Γ_Isp[J_AA, 0, AA_t[0]] * Γ_Isp[I_AA, 0, AA[1]] * Γ_Isp[J_AA, 0, AA_t[1]]
if I_BB.size > 0: 
    H_CI[I_BB, J_BB]  = H2[1, 1, BB[0], BB_t[0], BB[1], BB_t[1]]
    H_CI[I_BB, J_BB] *= Γ_Isp[I_BB, 1, BB[0]] * Γ_Isp[J_BB, 1, BB_t[0]] * Γ_Isp[I_BB, 1, BB[1]] * Γ_Isp[J_BB, 1, BB_t[1]]
if I_AB.size > 0:
    H_CI[I_AB, J_AB]  = H2[0, 1, AB[0], AB_t[0], AB[1], AB_t[1]]
    H_CI[I_AB, J_AB] *= Γ_Isp[I_AB, 0, AB[0]] * Γ_Isp[J_AB, 0, AB_t[0]] * Γ_Isp[I_AB, 1, AB[1]] * Γ_Isp[J_AB, 1, AB_t[1]]

In [ ]:
asd = '''
def helloworld():
    return "helloworld"
'''

SlaterCondon??

In [ ]:
exec(asd)

helloworld()

'helloworld'